In [5]:
from imdb import IMDb
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import cPickle as pickle
ia = IMDb(accessSystem='http')
from collections import defaultdict 
import io
from sklearn.linear_model import LogisticRegression, ElasticNet
import sklearn.svm

## Import and Split Data

In [6]:
mask=np.ones(dftouse.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)

## Model Setup
Definte do_classify and cv_optimize, used from homework 3.

In [ ]:
def cv_optimize(clf, parameters, X, y, n_folds=5, score_func=None):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, scoring=score_func)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds)
    gs.fit(X, y)
    print "BEST", gs.best_params_, gs.best_score_, gs.grid_scores_
    best = gs.best_estimator_
    return best

def do_classify(clf, parameters, indf, featurenames, targetname, target1val, mask=None, reuse_split=None, score_func=None, n_folds=5):
    subdf=indf[featurenames]
    X=subdf.values
    y=(indf[targetname].values==target1val)*1
    if mask !=None:
        print "using mask"
        Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
    if reuse_split !=None:
        print "using reuse split"
        Xtrain, Xtest, ytrain, ytest = reuse_split['Xtrain'], reuse_split['Xtest'], reuse_split['ytrain'], reuse_split['ytest']
    if parameters:
        clf = cv_optimize(clf, parameters, Xtrain, ytrain, n_folds=n_folds, score_func=score_func)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    test_accuracy = clf.score(Xtest, ytest)
    print "############# based on standard predict ################"
    print "Accuracy on training data: %0.3f" % (training_accuracy)
    print "Accuracy on test data:     %0.3f" % (test_accuracy)
    print confusion_matrix(ytest, clf.predict(Xtest))
    print "########################################################"
    return clf, Xtrain, ytrain, Xtest, ytest

## Logistic Regression Techniques

#### Logistic Ridge Regression



In [ ]:
clflog_ridge, Xtrain, ytrain, Xtest, ytest = do_classify(LogisticRegression(penalty="l1", solver="liblinear"),
                                                   {"C": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, 
                                                   indf=dftouse, features=lcols, targetname='winner', 1, mask=mask)

#### Logistic Regression with LASSO

In [ ]:
clflog_lasso, _, _, _, _ = do_classify(LogisticRegression(penalty="l2", solver="sag"),
                                       {"C": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, 
                                       indf=dftouse, features=lcols, targetname='winner', 1, reuse_split=reuse_split)

#### Elastic Net Regularized Logistic Regression

In [ ]:
clflog_elastic, _, _, _, _ = do_classify(ElasticNet, {"alpha": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
                                                      "l1_ratio": [0.2, 0.4, 0.6, 0.8]},
                                         indf=dftouse, features=lcols, targetname='winner', 1, reuse_split=reuse_split)

## SVM Techniques

#### SVM (Linear Kernel) - All Features Balanced

In [ ]:
BALANCE FEATURES IN CODE HERE

In [ ]:
clflog_lasso, _, _, _, _ = do_classify(LinearSVC(loss="hinge"), {"C": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, 
                                       indf=dftouse, features=lcols, targetname='winner', 1, reuse_split=reuse_split)


#### SVM (RBF Kernel) - All Features Balanced

In [ ]:
clflog_lasso, _, _, _, _ = do_classify(SVC(kernel="rbf"),
                                       {"C": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, 
                                       indf=dftouse, features=lcols, targetname='winner', 1, reuse_split=reuse_split)

#### SVM (Sigmoid Kernel) - All Features Balanced